In [2]:
import cobra
import pandas as pd
import os
from os.path import join
from cobra import Model, Reaction, Metabolite
from cobra.sampling import sampling
import numpy as np
os.environ["R_HOME"] = f"{os.environ['CONDA_PREFIX']}\\Lib\\R"
import rpy2.robjects
from plotnine import *
import matplotlib.pyplot as plt

In [3]:
# Importing the model
model=cobra.io.read_sbml_model("C:\\Users\\Maive\\Desktop\\BSc_loputoo\\Model_files\\Rt_IFO0880.xml")
model.objective = "BIOMASS_RT" 


Set parameter Username
Academic license - for non-commercial use only - expires 2025-01-21


In [4]:
# Lab data
glucose_uptakes= [0.476, 1.114, 1.648, 2.305, 2.6619851, 3.1] # the fifth glc uptake value was calculated
solution = model.optimize()
# All fluxes
all_fluxes = solution.fluxes.to_frame(name='Flux')

In [7]:
# Get all fluxes on different glucose uptake
all_fluxes_dif_glucose_uptakes = pd.DataFrame(columns=['Glucose uptake', *all_fluxes.index], index=range(len(glucose_uptakes))) #flux_values.index gives the row names column, * extracts the list of strings
biomass_GR = []

for i in range(len(glucose_uptakes)):
    medium = model.medium
    medium["EX_glc__D_e"] = glucose_uptakes[i]
    model.medium = medium
    
    solution = model.optimize()
    all_fluxes_dif_glucose_uptakes.loc[i] = solution.fluxes[['EX_glc__D_e', *all_fluxes.index]].values
    biomass_GR += [solution.objective_value]
all_fluxes_dif_glucose_uptakes.insert(0, 'Biomass growth rate', biomass_GR, True)

all_fluxes_dif_glucose_uptakes

# model.summary()


,Biomass growth rate,Glucose uptake,ALCD25yi,MTHFCm,AMPN,DAGCPTer_RT,PYRt2,NNDPRm,HMGCOASm,PDE4,...,EX_2hxmp_e,SALCNHe,EX_btn_e,BTNt2i,EX_fol_e,FOLt,NADtm,EX_pydxn_e,PYDXNtr,RIBFLVt2
0,0.033713,-0.476,0.0,0.0,0.0,0.0,0.0,0.0,-0.003255,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.081992,-1.114,0.0,0.0,0.0,0.0,0.0,0.0,-0.007917,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.122398,-1.648,0.0,0.0,0.0,0.0,0.0,0.0,-0.011818,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.172112,-2.305,0.0,0.0,0.0,0.0,0.0,0.0,-0.016619,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.199124,-2.661985,0.0,0.0,0.0,0.0,0.0,0.0,-0.019227,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.232267,-3.1,0.0,0.0,0.0,0.0,0.0,0.0,-0.022427,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
# Get specific metabolites with their fluxes
exchange_fluxes_table = all_fluxes_dif_glucose_uptakes[['BIOMASS_RT', 'EX_glc__D_e', 'EX_o2_e', 'EX_nh4_e','EX_so4_e','EX_pi_e', 'EX_co2_e']]
exchange_fluxes_table

,BIOMASS_RT,EX_glc__D_e,EX_o2_e,EX_nh4_e,EX_so4_e,EX_pi_e,EX_co2_e
0,0.033713,-0.476,-1.149964,-0.222516,-0.003379,-0.008975,1.276335
1,0.081992,-1.114,-2.534822,-0.541171,-0.008219,-0.021827,2.842163
2,0.122398,-1.648,-3.69407,-0.807865,-0.012269,-0.032584,4.152871
3,0.172112,-2.305,-5.120337,-1.135989,-0.017252,-0.045819,5.765485
4,0.199124,-2.661985,-5.895308,-1.314277,-0.01996,-0.05301,6.641709
5,0.232267,-3.1,-6.846184,-1.533033,-0.023282,-0.061833,7.716821


In [17]:
PPP_fluxes = all_fluxes_dif_glucose_uptakes[['BIOMASS_RT','G6PDH2r', 'TKT1', 'TALA','TKT2', 'XPK','PYRDC','PDHm']]
PPP_fluxes

,BIOMASS_RT,G6PDH2r,TKT1,TALA,TKT2,XPK,PYRDC,PDHm
0,0.033713,0.0,-0.076355,-0.076355,0.098377,0.0,0.015572,0.427713
1,0.081992,0.0,-0.1857,-0.1857,0.239258,0.0,0.077178,0.950726
2,0.122398,0.0,-0.277214,-0.277214,0.357167,0.0,0.120209,1.384278
3,0.172112,0.0,-0.389808,-0.389808,0.502234,0.0,0.173151,1.917693
4,0.199124,0.0,-0.450987,-0.450987,0.581057,0.0,0.201917,2.207528
5,0.232267,0.0,-0.526052,-0.526052,0.677772,0.0,0.237213,2.56315


In [18]:
ATPM_ACITL_fluxes = all_fluxes_dif_glucose_uptakes[['BIOMASS_RT','ATPM', 'ACITL']]
ATPM_ACITL_fluxes

,BIOMASS_RT,ATPM,ACITL
0,0.033713,1.22,0.0
1,0.081992,1.22,0.0
2,0.122398,1.22,0.0
3,0.172112,1.22,0.0
4,0.199124,1.22,0.0
5,0.232267,1.22,0.0


In [ ]:
# # Get all fluxes to excel
# with pd.ExcelWriter('C:\\Users\\Maive\\Desktop\\BSc_loputoo\\Results\\Simulated_fluxes\\Biomass_maximization\\all_fluxes_glc_uptake_005.xlsx') as excel_writer:
#     all_fluxes.to_excel(excel_writer, sheet_name='Glucose uptake = 0.476', index=True)

In [ ]:
# #Calculating the missing glucose uptake form lab data, when growth rate is 0.25
# growth_rates = np.array([0.049, 0.100, 0.151, 0.203, 0.25, 0.301]) 
# glucose_uptakes= np.array([-0.476, -1.114, -1.648, -2.305, -2.6619851, -3.1])

# Getting the missing glucose uptake when GR is 0.25
# from sklearn.linear_model import LinearRegression
# lm = LinearRegression()
# lm.fit(growth_rates[:, None], glucose_uptakes)

# y = lm.coef_ * 0.25 + lm.intercept_
# y


In [ ]:
# # Get all fluxes to excel
# with pd.ExcelWriter('C:\\Users\\Maive\\Desktop\\BSc_loputoo\\Results\\Simulated_fluxes\\flux_values_specific_glucose_uptake_all.xlsx') as excel_writer:
#     all_fluxes_dif_glucose_uptakes.to_excel(excel_writer, sheet_name='Sheet1', index=False)

# #  Get all flux values separately for dif growth rates, make them to a csv file
# for i in range(len(glucose_uptakes)):
#     all_fluxes_dif_glucose_uptakes.loc[i].to_csv(f'C:\\Users\\Maive\\Desktop\\BSc_loputoo\\Results\\Simulated_fluxes\\flux_values_specific_glucose_uptake_{glucose_uptakes[i]}.csv', index=True)

In [ ]:
# Getting exchange fluxes: glucose, CO2, ammonium and other minerals, others, like glycerol ['glc__D_e', 'o2_e', 'glyc_e', 'nh4_e',	'so4_e',	'pi_e', 'co2_e']  

# Make a pd dataframe with all exchange fluxes that are not zero, then make a pivot table with wanted metabolites fluxes on different growth rates

# model.reactions.EX_glc__D_e.bounds = glucose_uptakes[i], glucose_uptakes[i]
# solution = model.optimize('maximize')
# exchange_fluxes_all = model.summary().to_frame()
# exchange_fluxes_all['Glucose uptake'] = glucose_uptakes[0]


# for i in range(1, len(glucose_uptakes)):
#     model.reactions.EX_glc__D_e.bounds = glucose_uptakes[i], glucose_uptakes[i]
#     solution = model.optimize('maximize')
#     model_summary = model.summary().to_frame()
#     model_summary['Glucose uptake'] = glucose_uptakes[i]
#     exchange_fluxes_all = pd.concat([exchange_fluxes_all, model_summary], axis=0)  
    
# exchange_fluxes_all = exchange_fluxes_all[(exchange_fluxes_all['flux']) != 0.0] # for getting non-zero fluxes only
# exchange_fluxes_all['flux'] = abs(exchange_fluxes_all['flux'])
# exchange_fluxes_all

In [ ]:
# # Get all non-zero exchange fluxes to a pivot table
# exchange_fluxes_table_all = pd.pivot_table(exchange_fluxes_all, values='flux', index=['Glucose uptake'], columns=['reaction'])

# exchange_fluxes_table_all